In [1]:
import time
start = time.time()

In [2]:
import numpy as np # linear algebra
np.random.seed(42)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os # accessing directory structure
os.listdir('../input/fashion-dataset/fashion-dataset/')

['images.csv', 'images', 'styles.csv', 'styles']

In [3]:
## params
path = '../input/fashion-dataset/fashion-dataset/'
img_size = 128
batch_size = 64

In [4]:
df = pd.read_csv(path+"styles.csv", error_bad_lines=False)
df['image'] = df.id.apply(str) + '.jpg'
df.head()

b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg


In [5]:
'''sns.countplot(df.subCategory)
plt.title('Unique subCategory '+str(df.subCategory.nunique()))
plt.xticks(rotation = 90)'''

"sns.countplot(df.subCategory)\nplt.title('Unique subCategory '+str(df.subCategory.nunique()))\nplt.xticks(rotation = 90)"

In [6]:
print('before:',df.shape)
top10_subcats = df.subCategory.value_counts().sort_values(ascending=False).reset_index().head(10).iloc[:,0].tolist()
df = df[df.subCategory.isin(top10_subcats)]
print('after:',df.shape)

before: (44424, 11)
after: (36970, 11)


In [7]:
df_sub = df[['subCategory','image']]
# df_sub['image'] = path + 'images/' +df_sub.image
df_sub.head()

,subCategory,image
0,Topwear,15970.jpg
1,Bottomwear,39386.jpg
2,Watches,59263.jpg
3,Bottomwear,21379.jpg
4,Topwear,53759.jpg


In [8]:
from sklearn.model_selection import train_test_split, KFold

X_train, X_test, y_train, y_test = train_test_split(df_sub, df_sub.image, test_size = 0.2, random_state=42)
X_train.shape, X_test.shape

((29576, 2), (7394, 2))

In [9]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1/255.,
    # featurewise_center=True,
    # featurewise_std_normalization=True,
    rotation_range=20,
    zoom_range=0.2,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    horizontal_flip=True)


train_flow = train_datagen.flow_from_dataframe(X_train, directory=path+'/images/', 
                                    x_col='image', y_col='subCategory', 
                                    target_size=(img_size, img_size), 
                                    color_mode='grayscale', 
                                    classes=top10_subcats, 
                                    class_mode='categorical', 
                                    batch_size=batch_size, 
                                    shuffle=False, seed=42, 
                                    interpolation='nearest', drop_duplicates=False)

test_datagen = ImageDataGenerator(
    rescale=1/255.,
    # featurewise_center=True,
    # featurewise_std_normalization=True,
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # horizontal_flip=True
)


test_flow = test_datagen.flow_from_dataframe(X_test, directory=path+'/images/', 
                                    x_col='image', y_col='subCategory', 
                                    target_size=(img_size, img_size), 
                                    color_mode='grayscale', 
                                    classes=top10_subcats, 
                                    class_mode='categorical', 
                                    batch_size=batch_size, 
                                    shuffle=False, seed=42)

Using TensorFlow backend.


Found 29572 images belonging to 10 classes.
Found 7393 images belonging to 10 classes.


In [10]:
STEP_SIZE_TRAIN=(train_flow.n//train_flow.batch_size) + 1
STEP_SIZE_VALID=(test_flow.n//test_flow.batch_size) + 1
STEP_SIZE_TRAIN, STEP_SIZE_VALID

(463, 116)

In [11]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Conv2D, MaxPooling2D, Input, Flatten, Concatenate

'''arch = MobileNetV2(input_shape=(img_size, img_size, 3), 
                   include_top=False, 
                   weights=None, 
                   pooling=None)
dense = arch.output
dense = GlobalAveragePooling2D()(dense)
dense = Dense(512, activation='relu')(dense)
dense = Dense(10, activation='softmax')(dense)'''

inputs = Input(shape=(img_size, img_size, 1))
bn = BatchNormalization()(inputs)

conv = Conv2D(32, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu')(bn)
conv = Conv2D(32, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu')(conv)
conv = MaxPooling2D(pool_size=(2, 2), strides=(1, 1))(conv)

conv = Conv2D(64, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu')(conv)
conv = Conv2D(64, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu')(conv)
conv = MaxPooling2D(pool_size=(2, 2), strides=(1, 1))(conv)

conv = Conv2D(128, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu')(conv)
conv = Conv2D(128, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu')(conv)
conv = MaxPooling2D(pool_size=(2, 2), strides=(1, 1))(conv)

conv = Conv2D(256, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu')(conv)
conv = Conv2D(256, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu')(conv)
conv = MaxPooling2D(pool_size=(2, 2), strides=(1, 1))(conv)

pool = GlobalAveragePooling2D()(conv)

dense = Dense(256, activation='relu')(pool)
dense = Dense(10, activation='softmax')(dense)

cnn = Model(inputs=inputs, outputs=dense)

cnn.summary()

cnn.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='sgd')

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 1)       4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 126, 32)      320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 123, 123, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 121, 121, 64)      18496     
_________________________________________________________________
conv

In [13]:
from keras.callbacks import EarlyStopping, CSVLogger

es = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
log = CSVLogger('./train_performance.csv', separator=',', append=True)

cbs = [#es, 
       log
      ]

In [ ]:
train_history = cnn.fit_generator(train_flow, epochs=50, steps_per_epoch=STEP_SIZE_TRAIN, 
                                    validation_data=test_flow, validation_steps = STEP_SIZE_VALID, 
                                    callbacks = cbs)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50


In [ ]:
plt.plot(train_history.history['loss'])
plt.plot(train_history.history['val_loss'])

In [ ]:
plt.plot(train_history.history['acc'])
plt.plot(train_history.history['val_acc'])

In [ ]:
preds = cnn.predict_generator(test_flow, steps=STEP_SIZE_VALID, verbose = 1)
preds.shape

In [ ]:
subcat_lkup = dict((v,k) for k,v in test_flow.class_indices.items())

In [ ]:
preds_i = np.argmax(preds, axis=1)
preds_i.shape

In [ ]:
preds_fin = preds_i.tolist()
preds_fin = [subcat_lkup.get(i) for i in preds_fin]
acts_fin = [subcat_lkup.get(i) for i in test_flow.classes]

In [ ]:
from sklearn.metrics import confusion_matrix

plt.imshow(confusion_matrix(acts_fin, preds_fin))
plt.xticks(ticks=np.arange(10), labels=[v for k,v in subcat_lkup.items()], rotation=90)
plt.yticks(ticks=np.arange(10), labels=[v for k,v in subcat_lkup.items()])